### Temp - Will be convereted into main.py once complete
Currently being used for testing

In [1]:
import torch
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from utils.model_train import train

from utils.processing import load_data, augment_data, to_tensors, split_batch, process_data, incremental_save
from utils.quickdraw_cnn import QuickDrawCNN_V1, QuickDrawCNN_V2

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
image_dir = "data/numpy_bitmap/"

Device: cuda


In [3]:
categories = os.listdir(image_dir)
p1, p2 = categories[:len(categories)//2], categories[len(categories)//2:]

In [4]:
features_p1, labels_p1 = load_data(p1, image_dir, file_standardize=False)
features_p2, labels_p2 = load_data(p2, image_dir, file_standardize=False)

Loading 172 categories...


Loading data: 100%|██████████| 172/172 [07:30<00:00,  2.62s/it]


Data loaded, size: (25234905, 784)
Loading 173 categories...


Loading data: 100%|██████████| 173/173 [07:32<00:00,  2.61s/it]


Data loaded, size: (25191361, 784)


In [9]:
# features, labels = process_data(features_p1, features_p2, labels_p1, labels_p2, portion=0.2)
features_p1 = features_p1.reshape(-1, 28, 28)
features_p2 = features_p2.reshape(-1, 28, 28)

labels_p1 = [label.split(".")[0] for label in labels_p1]
labels_p2 = [label.split(".")[0] for label in labels_p2]

In [10]:
labels = labels_p1 + labels_p2
labels_map = {label: i for i, label in enumerate(set(labels))}

In [12]:
def to_tensors_split(features: np.array, labels: np.array, device: str = 'cpu', labels_map: dict = None):
    '''
    Normalize objects convert the features and labels into tensor objects
    
    args:
    - features: numpy array with the features
    - labels: numpy array with the labels
    - device: device to use
    
    returns: features, labels
    '''
    
    print(f"Converting {len(labels)} images to tensors, using {device}...")
            
    labels = [labels_map[label] for label in labels] # convert labels to integers
    
    features = torch.tensor(features).to(device) / 255.0 # normalizing 0-255 -> 0-1
    features = features.unsqueeze(1) # adding channel dimension for CNN, 1 channel for grayscale
    labels = torch.tensor(labels).to(device)
    
    print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    
    return features, labels

In [ ]:
features_p1, labels_p1 = to_tensors_split(features_p1, labels_p1, device, labels_map)
features_p2, labels_p2 = to_tensors_split(features_p2, labels_p2, device, labels_map)

Converting 25234905 images to tensors, using cuda...


In [ ]:
train_loader, test_loader, val_loader = split_batch(features_p1, labels_p1, batch_size=64)

Splitting torch.Size([607624, 1, 28, 28]) images into train, test, and validation sets...
Train size: 486099 
Validation size: 60762 
Test size: 60763


In [8]:
model = QuickDrawCNN_V2().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
train_loss, val_loss, train_acc, val_acc = train(model = model,
                                                  train_loader = train_loader,
                                                  val_loader = val_loader,
                                                  epochs = 10,
                                                  criterion = criterion,
                                                  optimizer = optimizer,
                                                  device = device)   


Epoch 1/10


Training:  10%|█         | 781/7596 [00:02<00:24, 276.88batch/s, acc=89.1, loss=0.31] 


KeyboardInterrupt: 

In [29]:
parent_path = 'saves'

varaibles_saved = incremental_save(f'{parent_path}/vars/qd_cnn_v2')
with open(varaibles_saved, "wb") as f:
    pickle.dump(labels_map, f)
    pickle.dump(train_loss, f)
    pickle.dump(val_loss, f)
    pickle.dump(train_acc, f)
    pickle.dump(val_acc, f)

model_saved = incremental_save(f'{parent_path}/model/qd_cnn_v2', data=model)

Path: saves/vars/qd_cnn_v2.0.pth
Saved to saves/model/qd_cnn_v2.0.pth.
